In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!apt-get update > /dev/null 2>&1
!pip install selenium > /dev/null 2>&1
!apt install chromium-chromedriver > /dev/null 2>&1

In [3]:
#각종 모듈 import
import time # 새로운 페이지로 셀레니음 웹드라이버가 넘어갈 때마다 잠시 멈춰주는 역할을 위해 불러옴. 
# 멈춰주지 않으면 페이지가 다 로딩되기도 전에 다음 코드를 실행해버리는 불상사가 생김. 그걸 막아준다.
from selenium import webdriver # 웹드라이버. 코랩이라 우리가 직접 볼 순 없지만 얘가 돌아다니면서 일을 해줌
from selenium.webdriver.common.keys import Keys # 이 파일에선 아직 딱히 쓰지 않은듯
from selenium.webdriver.common.by import By # element를 찾을 때 쓰이는 parameter를 불러오는 역할
import pandas as pd 
from bs4 import BeautifulSoup # html 요소에 접근해서 정보를 빼내올 때 씀
from tqdm import tqdm
import pickle
import numpy as np

In [4]:
import warnings # 사실 잘 모르지만 선생님이 하셨던거 암튼 가져옴
warnings.filterwarnings('ignore')

In [5]:
#크롬 드라이버의 각종 옵션들 생성 및 적용.
options = webdriver.ChromeOptions()
options.add_argument('--headless') # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument('--single-process')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36') # 나는 기계가 아니고 사람이에요
driver = webdriver.Chrome('chromedriver', options=options)

In [6]:
driver.get('https://blog.daum.net/chamsaemmul/18284627?category=222732')

In [9]:
text = driver.find_element(By.XPATH, '//*[@id="div_${article.articleNo}"]/div/p').text

In [ ]:
text.split('\n\n') # 한 문단에 담겼을 때에만 문맥이 생김. 그 문맥들을 모델에 학습시키기 위해 크롤링하는 것이니 일정 길이 미만이면 버리는 것도 좋다.
# 이 블로그에서 엔터 두번으로 문단을 나눔. 
# 작가의 이름은 필요가 없다. 버리자. 애초에 일정 길이 미만이니 버려질 것.
# 정규표현식으로 한글 외에는 날릴 것이다. 

In [15]:
text2 = text.split('\n\n')
xx = []
for i in text2:
    x = i.replace('\n', ' ')
    if len(x.split()) > 4:
        xx.append(x)
xx

['세월은 가고 세상은 더 헐벗으니 나는 노래를 불러야지 새들이 아직 하늘을 날 때',
 '아이들은 자라고 어른들은 늙어가니 나는 노래를 불러야지 사람들의 목소리가 들리는 동안',
 '무슨 터질 듯한 立場입장이 있겠느냐 항상 빗나가는 구실 무슨 거창한 목표가 있겠느냐 나는 그냥 노래를 부를 뿐 사람들이 서로 미워하는 동안',
 '나그네 흐를 길은 이런 거지 저런 거지 같이 가는 길 어느 길목이나 나무들은 서서 바람의 길잡이가 되고 있는데 나는 노래를 불러야지 사람들이 걸신걸신을 섬기는 동안',
 '하늘의 눈동자도 늘 보이고 땅의 눈동자도 보이니 나는 내 노래를 불러야지 우리가 여기 살고 있는 동안',
 '사람이 바다로 가서 바닷바람이 되어 불고 있다든지, 아주 추운데로 가서 눈으로 내리고 있다든지, 사람이 따뜻한 데로 가서 햇빛으로 빛나고 있다든지, 해지는 쪽으로 가서 황혼에 녹아 붉은 빛을 내고 있다든지 그 모양이 다 갈데 없이 아름답습니다.',
 '나무들을 열어놓는 새소리 풀잎들을 물들이는 새 소리의 푸른 그림자 내 머리 속 유리창을 닦는 심장의 창문을 열어놓는 새소리의 저 푸른 통로',
 '풀이여 푸른빛이여 감격해본지 얼마나 됐는지',
 '갈수록, 일월(日月)이여, 내 마음 더 여리어져 가는 8월을 견딜 수 없네. 9월도 시월도 견딜 수 없네. 흘러가는 것들을 견딜 수 없네. 사람의 일들 변화와 아픔들을 견딜 수 없네. 있다가 없는 것 보이다 안 보이는 것 견딜 수 없네. 시간을 견딜 수 없네. 시간의 모든 흔적들 그림자들 견딜 수 없네. 모든 흔적은 상흔(傷痕)이니 흐르고 변하는 것들이여 아프고 아픈 것들이여.',
 '불행의 대부분은 경청할 줄 몰라서 그렇게 되는 듯. 비극의 대부분은 경청하지 않아서 그렇게 되는 듯. 아, 오늘날처럼 경청이 필요한 때는 없는 듯. 대통령이든 신(神)이든 어른이든 애이든 아저씨든 아줌마든 무슨 소리이든지 간에 내 안팎의 소리를 경청할 줄 알면 세상이 조금은 좋아질 듯. 모든 귀가 막혀 있어 우리의 행성은 캄캄하고 기가 막

In [ ]:
driver.find_element(By.XPATH, '//*[@id="cContentBody"]').text.split('\n\n')

In [17]:
url = 'https://blog.daum.net/chamsaemmul/18284631?category=222732'
driver.get(url)
sentence = []
for _ in range(2):
    text = driver.find_element(By.XPATH, '//*[@id="cContentBody"]').text.split('\n\n')
    for i in text:
        t = i.replace('\n', ' ')
        if len(t.split()) > 4:
            sentence.append(t)
    driver.find_element(By.XPATH, '//*[@id="cNumbering"]/a[2]').click()
    time.sleep(1.5)

In [18]:
sentence

[' 지금은 남의 땅 - 빼앗긴 들에도 봄은 오는가?',
 ' 나는 온몸에 햇살을 받고 푸른 하늘 푸른 들이 맞붙은 곳으로 가르마 같은 논길을 따라 꿈 속을 가듯 걸어만 간다.',
 ' 입술을 다문 하늘아, 들아, 내 맘에는 내 혼자 온 것 같지를 않구나! 네가 끌었느냐, 누가 부르더냐. 답답워라, 말을 해 다오.',
 ' 바람은 내 귀에 속삭이며 한 자욱도 섰지 마라, 옷자락을 흔들고. 종다리는 울타리 너머 아씨같이 구름 뒤에서 반갑다 웃네.',
 ' 고맙게 잘 자란 보리밭아, 간밤 자정이 넘어 내리던 고운 비로 너는 삼단 같은 머리털을 감았구나, 내 머리조차 가뿐하다.',
 ' 혼자라도 가쁘게나 가자. 마른 논을 안고 도는 착한 도랑이 젖먹이 달래는 노래를 하고, 제 혼자 어깨춤만 추고 가네.',
 ' 나비 제비야 깝치지 마라. 맨드라미 들마꽃에도 인사를 해야지. 아주까리 기름을 바른 이가 지심 매던 그 들이라 다 보고 싶다.',
 ' 내 손에 호미를 쥐어 다오. 살진 젖가슴과 같은 부드러운 이 흙을 발목이 시도록 밟아도 보고, 좋은 땀조차 흘리고 싶다.',
 ' 강가에 나온 아이와 같이, 짬도 모르고 끝도 없이 닫는 내 혼아 무엇을 찾느냐, 어디로 가느냐, 웃어웁다, 답을 하려무나.',
 ' 나는 온몸에 풋내를 띠고, 푸른 웃음 푸른 설움이 어우러진 사이로 다리를 절며 하루를 걷는다. 아마도 봄 신령이 지폈나 보다.',
 ' 그러나, 지금은 - 들을 빼앗겨 봄조차 빼앗기겠네.',
 ' 2.<나의 침실로> "가장 아름답고 오__랜 것은 오직 꿈 속에만 있어라" __\'내말\'',
 " '마돈나' 지금은 밤도, 모든 목거지에, 다니노라 피곤하여 돌아 가련도다 아,너도 먼동이 트기 전으로, 수밀도(水蜜桃)의 네 가슴에, 이슬이 맺도록 달려오너라.",
 " '마돈나' 오려무나, 네 집에서 눈으로 유전(遺傳)하던 진주는, 다 두고 몸만 오너라. 빨리 가자, 우리는 밝음이 오면, 어딘지도 모르게 숨는 두 별이어라. '마돈나' 구석지고도 어두운 마음의 거리에서, 나는

In [19]:
df = pd.DataFrame(sentence, columns = ['시'])
df

,시
0,지금은 남의 땅 - 빼앗긴 들에도 봄은 오는가?
1,나는 온몸에 햇살을 받고 푸른 하늘 푸른 들이 맞붙은 곳으로 가르마 같은 논길을 ...
2,"입술을 다문 하늘아, 들아, 내 맘에는 내 혼자 온 것 같지를 않구나! 네가 끌었..."
3,"바람은 내 귀에 속삭이며 한 자욱도 섰지 마라, 옷자락을 흔들고. 종다리는 울타리..."
4,"고맙게 잘 자란 보리밭아, 간밤 자정이 넘어 내리던 고운 비로 너는 삼단 같은 머..."
...,...
83,현재에서 미래로 아파트에 눌려 산도 가고 물도 갔다
84,화신 등진 저 아낙네들 지나간 고운 날을 삼키며 쑥을 캐는 눈시울이 따가워선가 가난...
85,저렇게 많은 별 중에서 별 하나가 나를 내려다본다 이렇게 많은 사람 중에서 그 별 ...
86,밤이 깊을수록 별은 밝음 속에 사라지고 나는 어둠 속에 사라진다


In [20]:
url = 'https://blog.daum.net/chamsaemmul/18284631?category=222732'
driver.get(url)
sentence = []
for _ in tqdm(range(101)):
    text = driver.find_element(By.XPATH, '//*[@id="cContentBody"]').text.split('\n\n')
    for i in text:
        t = i.replace('\n', ' ')
        if len(t.split()) > 4:
            sentence.append(t)
    driver.find_element(By.XPATH, '//*[@id="cNumbering"]/a[2]').click()
    time.sleep(1.5)

100%|██████████| 101/101 [04:48<00:00,  2.86s/it]


In [21]:
df = pd.DataFrame(sentence, columns= ['lyric'])

In [22]:
df

,lyric
0,지금은 남의 땅 - 빼앗긴 들에도 봄은 오는가?
1,나는 온몸에 햇살을 받고 푸른 하늘 푸른 들이 맞붙은 곳으로 가르마 같은 논길을 ...
2,"입술을 다문 하늘아, 들아, 내 맘에는 내 혼자 온 것 같지를 않구나! 네가 끌었..."
3,"바람은 내 귀에 속삭이며 한 자욱도 섰지 마라, 옷자락을 흔들고. 종다리는 울타리..."
4,"고맙게 잘 자란 보리밭아, 간밤 자정이 넘어 내리던 고운 비로 너는 삼단 같은 머..."
...,...
9771,"다시는, 나의 이러한 맘뿐은, 때가 되면, 그림자 같이 당신한테로 가오리다."
9772,그립다 말을 할까 하니 그리워.
9773,그냥 갈까 그래도 다시 한번...
9774,"저 산에도 까마귀, 들에 까마귀 서산에는 해 진다고 지저귑니다."


In [35]:
df2 = df['lyric'].str.replace(' 좋아요공감 공유하기 글 요소 저작자표시비영리변경금지', '')
df2[28]

'   '

In [ ]:
'좋아요공감 공유하기 글 요소 저작자표시비영리변경금지' #없애야 함

In [36]:
df['lyric'] = df['lyric'].str.replace('좋아요공감 공유하기 글 요소 저작자표시비영리변경금지', '')

In [38]:
df['lyric'] = df['lyric'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '')
df['lyric'] = df['lyric'].str.replace('^ +', '')
df['lyric'] = df['lyric'].replace('', np.nan)

In [40]:
df.isnull().sum()

lyric    1
dtype: int64

In [41]:
df

,lyric
0,지금은 남의 땅 빼앗긴 들에도 봄은 오는가
1,나는 온몸에 햇살을 받고 푸른 하늘 푸른 들이 맞붙은 곳으로 가르마 같은 논길을 따...
2,입술을 다문 하늘아 들아 내 맘에는 내 혼자 온 것 같지를 않구나 네가 끌었느냐 누...
3,바람은 내 귀에 속삭이며 한 자욱도 섰지 마라 옷자락을 흔들고 종다리는 울타리 너머...
4,고맙게 잘 자란 보리밭아 간밤 자정이 넘어 내리던 고운 비로 너는 삼단 같은 머리털...
...,...
9771,다시는 나의 이러한 맘뿐은 때가 되면 그림자 같이 당신한테로 가오리다
9772,그립다 말을 할까 하니 그리워
9773,그냥 갈까 그래도 다시 한번
9774,저 산에도 까마귀 들에 까마귀 서산에는 해 진다고 지저귑니다


In [42]:
df.drop_duplicates(subset=['lyric'], inplace=True)
df.dropna(how='any', inplace=True)

In [43]:
df.isnull().sum(), len(df)

(lyric    0
 dtype: int64, 9487)

In [44]:
df.head()

,lyric
0,지금은 남의 땅 빼앗긴 들에도 봄은 오는가
1,나는 온몸에 햇살을 받고 푸른 하늘 푸른 들이 맞붙은 곳으로 가르마 같은 논길을 따...
2,입술을 다문 하늘아 들아 내 맘에는 내 혼자 온 것 같지를 않구나 네가 끌었느냐 누...
3,바람은 내 귀에 속삭이며 한 자욱도 섰지 마라 옷자락을 흔들고 종다리는 울타리 너머...
4,고맙게 잘 자란 보리밭아 간밤 자정이 넘어 내리던 고운 비로 너는 삼단 같은 머리털...


In [46]:
df.tail()

,lyric
9771,다시는 나의 이러한 맘뿐은 때가 되면 그림자 같이 당신한테로 가오리다
9772,그립다 말을 할까 하니 그리워
9773,그냥 갈까 그래도 다시 한번
9774,저 산에도 까마귀 들에 까마귀 서산에는 해 진다고 지저귑니다
9775,앞 강물 뒷 강물 흐르는 물은 어서 따라 오라고 따라 가자고 흘러도 연달아 흐릅디다려


In [50]:
df.to_csv('poetry.csv', index=False)